In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import requests
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langsmith import traceable
from serpapi import GoogleSearch

load_dotenv()

## Single tool

In [ ]:
model = ChatOpenAI(model_name="gpt-4.1-mini")


@tool
def get_weather(latitude, longitude):
    """Get the weather of a given latitude and longitude"""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]["temperature_2m"]


tools_mapping = {
    "get_weather": get_weather,
}

model_with_tools = model.bind_tools([get_weather])


@traceable
def get_response(question: str):
    messages = [
        SystemMessage(
            "You're a helpful assistant. Use the tools provided when relevant."
        ),
        HumanMessage(question),
    ]
    ai_message = model_with_tools.invoke(messages)
    messages.append(ai_message)

    for tool_call in ai_message.tool_calls:
        selected_tool = tools_mapping[tool_call["name"]]
        tool_msg = selected_tool.invoke(tool_call)
        messages.append(tool_msg)

    ai_message = model_with_tools.invoke(messages)
    messages.append(ai_message)

    return ai_message.content


response = get_response("What is the temperature in Madrid?")
print(response)

## Multiple tools 

In [ ]:
model = ChatOpenAI(model_name="gpt-4.1-mini")


@tool
def get_weather(latitude, longitude):
    """Get the weather of a given latitude and longitude"""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]["temperature_2m"]


@tool
def check_guidelines(drafted_response: str) -> str:
    """Check if a given response follows the company guidelines"""
    model = ChatOpenAI(model_name="gpt-4.1-mini")
    response = model.invoke(
        [
            SystemMessage(
                "You're a helpful assistant. Your task is to check if a given response follows the company guidelines. The company guidelines are that responses should be written in the style of a haiku. You should reply with 'OK' or 'REQUIRES FIXING' and a short explanation."
            ),
            HumanMessage(f"Current response: {drafted_response}"),
        ]
    )
    return response.content


tools_mapping = {
    "get_weather": get_weather,
    "check_guidelines": check_guidelines,
}

model_with_tools = model.bind_tools([get_weather, check_guidelines])


@traceable
def get_response(question: str):
    messages = [
        SystemMessage(
            "You're a helpful assistant. Use the tools provided when relevant. Then draft a response and check if it follows the company guidelines. Only respond to the user after you've validated and modified the response if needed."
        ),
        HumanMessage(question),
    ]
    ai_message = model_with_tools.invoke(messages)
    messages.append(ai_message)

    while ai_message.tool_calls:
        for tool_call in ai_message.tool_calls:
            selected_tool = tools_mapping[tool_call["name"]]
            tool_msg = selected_tool.invoke(tool_call)
            messages.append(tool_msg)
        ai_message = model_with_tools.invoke(messages)
        messages.append(ai_message)

    return ai_message.content


response = get_response("What is the temperature in Madrid?")
print(response)

## Exercise:

Build a function calling workflow that let users get the latest news from multiple companies and groups them according to their topic 